In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
import os

# resolution analysis

In [162]:
# import all result data 
with open('V101/ORB/data/results.txt') as json_file:
    data_100 = json.load(json_file)

with open('V1010.8/ORB/data/results.txt') as json_file:
    data_80 = json.load(json_file)
    
with open('V1010.6/ORB/data/results.txt') as json_file:
    data_60 = json.load(json_file)
    
with open('V1010.4/ORB/data/results.txt') as json_file:
    data_40 = json.load(json_file)
    
with open('V101/DSM/data/results.txt') as json_file:
    data_dsm = json.load(json_file)

In [35]:
filenames = pd.read_csv("../config/data_meta.csv", sep=";").filename[0:9].tolist()

dataset_paths = [os.path.join("..", "results", filename) for filename in filenames]

result_paths_orb = [os.path.join(data_path, "ORB", "data") for data_path in dataset_paths]
result_paths_dsm = [os.path.join(data_path, "DSM", "data") for data_path in dataset_paths]
result_paths_dso = [os.path.join(data_path, "DSO", "data") for data_path in dataset_paths]


# trajectory analysis



In [53]:
np.load(os.path.join(result_paths_dso[0], "pos_dist_to_gt.npy"))

# distances
dists_orb = np.concatenate([np.load(os.path.join(result_path_orb, "pos_dist_to_gt.npy")) for result_path_orb in result_paths_orb])
dists_dsm = np.concatenate([np.load(os.path.join(result_path_dsm, "pos_dist_to_gt.npy")) for result_path_dsm in result_paths_dsm])
dists_dso = np.concatenate([np.load(os.path.join(result_path_dso, "pos_dist_to_gt.npy")) for result_path_dso in result_paths_dso])

(21481,)
(2713,)
(5219,)


In [55]:
plot = go.Figure(layout_yaxis_range=[0,1.5])
  
plot.add_trace(go.Box(y=dists_orb, name="ORB"))
plot.add_trace(go.Box(y=dists_dsm, name="DSM"))
plot.add_trace(go.Box(y=dists_dso, name="DSO"))

  
plot.show()

ValueError: Image generation requires the psutil package.

Install using pip:
    $ pip install psutil

Install using conda:
    $ conda install psutil


In [64]:
fig = go.Figure(data=go.Scatter(x=np.array(range(len(dists_orb))), y=dists_orb))
fig.show()

In [173]:
dsm_proc_time = data_dsm["processing_time"]
orb_proc_time = processing_times[0]
import plotly.express as px
import numpy

df = pd.DataFrame({"Algorithm": ["orb", "dsm"],  "processing_time":[orb_proc_time -15, dsm_proc_time -15]})

fig = px.bar(df, x = "Algorithm", y = "processing_time")
fig.update_layout(
    title="Processing time of a single sequence (140s in length (20fps))"
)
fig.show()

In [105]:
fig = go.Figure(data=go.Scatter(x=[100, 80, 60, 40], y=processing_times))
fig.update_layout(
    title="Influence of pixel count on the calculation time",
    xaxis_title="percentage of pixel count of original resolution (752 x 480)",
    yaxis_title="calculation time in sec"
)
fig.show()

In [106]:
orb_est_df_100 = pd.read_csv("V101/ORB/data/est_df_transformed.csv")
dsm_est_df_100 = pd.read_csv("V101/DSM/data/est_df_transformed.csv")
gt = pd.read_csv("../tmp/data.csv")
gt["timestamp"] = gt["#timestamp"]
gt["p_x"] = gt[" p_RS_R_x [m]"]
gt["p_y"] = gt[" p_RS_R_y [m]"]
gt["p_z"] = gt[" p_RS_R_z [m]"]


In [107]:
class FramePreprocessor:
    def __init__(self, gt_filepath, est_filepath, dataset_type, outdir_data, gt_pos_df=None, est_pos_df=None):
        self.dataset_type = dataset_type
        self.gt_filepath = gt_filepath
        self.est_filepath = est_filepath
        self.outdir_data = outdir_data
        self.gt_pos_df = gt_pos_df
        self.est_pos_df = est_pos_df

    @staticmethod
    def standardize_timestamp(ts):
        ts = '{0:.10f}'.format(ts)
        ts_head = ts[0:10]
        if ts[10] == ".":
            ts_tail = ts[11:14]
        else:
            ts_tail = ts[10:13]
        ts_out = ts_head + "." + ts_tail
        return float(ts_out)

    def create_gt_pos_df(self):
        if self.dataset_type == "euroc":
            gt_df = pd.read_csv(self.gt_filepath, sep=",")
            
            # for now only use positional dataset_type
            gt_pos_df = pd.DataFrame({"timestamp": gt_df["#timestamp"],
                                      "p_x": gt_df[" p_RS_R_x [m]"],
                                      "p_y": gt_df[" p_RS_R_y [m]"],
                                      "p_z": gt_df[" p_RS_R_z [m]"]})
            gt_pos_df.timestamp = gt_pos_df.timestamp.apply(self.__class__.standardize_timestamp)
            self.gt_pos_df = gt_pos_df
            
    def create_est_pos_df(self):
        def standardize_seps(filepath, target_sep, invalid_seps):
            lines_out = []
            file = open(filepath, "r+")
            lines = file.readlines()
            for line in lines:
                for invalid_sep in invalid_seps:
                    line = line.replace(invalid_sep, target_sep)
                lines_out.append(line)
            file.truncate(0)
            file.close()
            file = open(filepath, "w")
            file.writelines(lines_out)
            file.close()

        standardize_seps(invalid_seps=["    ", "   ", "  "],
                         target_sep=" ",
                         filepath=self.est_filepath)

        if self.dataset_type == "euroc":
            df_calc = pd.read_csv(self.est_filepath,
                                  sep=" ",
                                  header=None,
                                  names=["timestamp", "p_x", "p_y", "p_z", "1", "2", "3", "4"])
            df_calc.timestamp = df_calc.timestamp.apply(self.__class__.standardize_timestamp)
            self.est_pos_df = df_calc[["timestamp", "p_x", "p_y", "p_z"]]

    def align_timestamps(self):
        if all([isinstance(self.est_pos_df, pd.DataFrame), isinstance(self.gt_pos_df, pd.DataFrame)]):
            gt_ts = self.gt_pos_df["timestamp"].to_numpy()
            est_ts = self.est_pos_df["timestamp"].to_numpy()
            if not (np.all(np.diff(est_ts) >= 0) and np.all(np.diff(gt_ts) >= 0)):
                raise ValueError("Timestamps have to be sorted!")

            # remove all rows in the estimated data, where no groundtruth is present
            perc_no_gt = ((np.sum(est_ts < gt_ts[0]) + np.sum(est_ts > gt_ts[len(gt_ts) - 1])) / len(
                est_ts) * 100)
            if perc_no_gt != 0:

                print(str(perc_no_gt) + " percent of measurements countain no groundtruth, they will be removed!")
                idx_oi = [((est_ts[i] > gt_ts[0]) and (est_ts[i] < gt_ts[len(gt_ts) - 1])).tolist() for i in
                          range(len(est_ts))]
                est_ts = est_ts[idx_oi]
                self.est_pos_df = self.est_pos_df[idx_oi]

            # for each estimated point, only use the closest point in
            # TODO: make this code less complex in time
            matches = {}
            for est_ts_oi in est_ts:
                dists = [abs(est_ts_oi - val) for val in gt_ts]
                idx_oi = np.argmin(dists)
                if dists[idx_oi] < 200000000:
                    matches[est_ts_oi] = gt_ts[idx_oi]
                else:
                    print("for timestamp {}, no matching groundtruth found!".format(est_ts_oi))
                    matches[est_ts_oi] = None
            self.gt_pos_df = self.gt_pos_df[[val in matches.values() for val in self.gt_pos_df["timestamp"]]]

           # if not self.gt_pos_df.shape == self.est_pos_df.shape:
            #    raise ValueError("Something went wrong, position dataframes of different dimension!")
        else:
            raise ValueError("No dataframes found!")

    def get_gt_pos_df(self):
        return self.gt_pos_df

    def get_est_pos_df(self):
        return self.est_pos_df


In [108]:
fp = FramePreprocessor(gt_filepath=None, est_pos_df=orb_est_df_100, outdir_data=None, gt_pos_df=gt, est_filepath=None, 
                      dataset_type="euroc")
fp.gt_pos_df.timestamp = fp.gt_pos_df.timestamp.apply(fp.standardize_timestamp)
fp.align_timestamps()

In [110]:
gt_long_orb = fp.get_gt_pos_df()
est_long_orb = fp.get_est_pos_df()
gt_long_orb.head()
est_long_orb.head()

,Unnamed: 0,timestamp,p_x,p_y,p_z
0,0,1.403715e+09,0.872252,2.209865,1.019889
1,1,1.403715e+09,0.879990,2.214823,1.039097
2,2,1.403715e+09,0.887048,2.218270,1.056643
3,3,1.403715e+09,0.894383,2.221724,1.072270
4,4,1.403715e+09,0.900127,2.223443,1.080486


In [111]:
class Evaluator:
    def __init__(self, gt_df, est_df):
        self.gt_df = gt_df
        self.est_df = est_df

    def calculate_diff(self):
        dists = []
        for i in range(self.est_df.shape[0]):
            p1 = np.array([self.est_df.iloc[i, 2], self.est_df.iloc[i, 3], self.est_df.iloc[i, 4]])
            p2 = np.array([self.gt_df.iloc[i, 1], self.gt_df.iloc[i, 2], self.gt_df.iloc[i, 3]])
            dist = np.linalg.norm(p1-p2)
            dists.append(dist)
        mean_diff = np.mean(dists)
        print("mean difference position error: {}".format(mean_diff))
        return dists, mean_diff

In [112]:
evaluator = Evaluator(gt_df=gt_long_orb, 
                      est_df=est_long_orb)

In [113]:
dists_orb, mean_diff_orb = evaluator.calculate_diff()

mean difference position error: 0.08256527734471082


In [114]:
fp = FramePreprocessor(gt_filepath=None, est_pos_df=dsm_est_df_100, outdir_data=None, gt_pos_df=gt, est_filepath=None, 
                      dataset_type="euroc")
fp.gt_pos_df.timestamp = fp.gt_pos_df.timestamp.apply(fp.standardize_timestamp)
fp.align_timestamps()
gt_long = fp.get_gt_pos_df()
est_long = fp.get_est_pos_df()
print(gt_long.head())
est_long.head()

               #timestamp   p_RS_R_x [m]   p_RS_R_y [m]   p_RS_R_z [m]  \
882   1403715278712142848       0.905660       2.153127       0.977794   
952   1403715279062142976       0.959404       2.179212       1.086272   
1252  1403715280562142976       1.067345       2.220378       1.209176   
1312  1403715280862142976       1.116619       2.252080       1.250248   
1442  1403715281512142848       1.227798       2.324085       1.297879   

       q_RS_w []   q_RS_x []   q_RS_y []   q_RS_z []   v_RS_R_x [m s^-1]  \
882     0.062580   -0.816324   -0.049765   -0.572034            0.145626   
952     0.065887   -0.808885   -0.047710   -0.582312            0.129306   
1252    0.044035   -0.821187   -0.046874   -0.567025            0.154169   
1312    0.016319   -0.819398    0.014443   -0.572814            0.166666   
1442    0.042114    0.815451   -0.096671    0.569142            0.177352   

       v_RS_R_y [m s^-1]  ...   b_w_RS_S_x [rad s^-1]   b_w_RS_S_y [rad s^-1]  \
882             0

,Unnamed: 0,timestamp,p_x,p_y,p_z
0,1,1.403715e+09,0.855608,2.197245,0.973882
1,2,1.403715e+09,0.907236,2.226212,1.080113
2,3,1.403715e+09,1.015739,2.264961,1.199039
3,4,1.403715e+09,1.076110,2.296096,1.247413
4,5,1.403715e+09,1.206199,2.367188,1.288267


In [92]:
evaluator = Evaluator(gt_df=gt_long, 
                      est_df=est_long)
dists_dsm, mean_diff_dsm = evaluator.calculate_diff()

mean difference position error: 0.08586807546021107


In [100]:
fig = go.Figure(data=go.Scatter(x=est_long.timestamp - est_long.timestamp[0], y=dists_dsm))
fig.update_layout(
    title="Influence of pixel count on the calculation time",
    xaxis_title="percentage of pixel count of original resolution (752 x 480)",
    yaxis_title="calculation time in sec"
)
fig.show()

In [116]:
fig = go.Figure(data=go.Scatter(x=est_long_orb.timestamp - est_long_orb.timestamp[0], y=dists_orb))
fig.update_layout(
    title="Influence of pixel count on the calculation time",
    xaxis_title="percentage of pixel count of original resolution (752 x 480)",
    yaxis_title="calculation time in sec",
    color="green"
)
fig.show()

ValueError: Invalid property specified for object of type plotly.graph_objs.Layout: 'color'

Did you mean "polar"?

    Valid properties:
        activeshape
            :class:`plotly.graph_objects.layout.Activeshape`
            instance or dict with compatible properties
        angularaxis
            :class:`plotly.graph_objects.layout.AngularAxis`
            instance or dict with compatible properties
        annotations
            A tuple of
            :class:`plotly.graph_objects.layout.Annotation`
            instances or dicts with compatible properties
        annotationdefaults
            When used in a template (as
            layout.template.layout.annotationdefaults), sets the
            default property values to use for elements of
            layout.annotations
        autosize
            Determines whether or not a layout width or height that
            has been left undefined by the user is initialized on
            each relayout. Note that, regardless of th

In [161]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=est_long_orb.timestamp - est_long_orb.timestamp[0], y=dists_orb,
                    mode='lines',
                    name='orb'))

# Add traces
fig.add_trace(go.Scatter(x=est_long.timestamp - est_long.timestamp[0], y=dists_dsm,
                    mode='lines',
                    name='dsm'))

fig.update_layout(
    title="Error of estimated position over time",
    xaxis_title="Time in s",
    yaxis_title="Euclidean distance to true position",
)

fig.show()

In [158]:
gt_long_orb

,#timestamp,p_RS_R_x [m],p_RS_R_y [m],p_RS_R_z [m],q_RS_w [],q_RS_x [],q_RS_y [],q_RS_z [],v_RS_R_x [m s^-1],v_RS_R_y [m s^-1],...,b_w_RS_S_x [rad s^-1],b_w_RS_S_y [rad s^-1],b_w_RS_S_z [rad s^-1],b_a_RS_S_x [m s^-2],b_a_RS_S_y [m s^-2],b_a_RS_S_z [m s^-2],timestamp,p_x,p_y,p_z
912,1403715278862142976,0.930110,2.165487,1.024433,0.067113,-0.810226,-0.046801,-0.580378,0.167395,0.079719,...,-0.002229,0.020700,0.07635,-0.012493,0.547731,0.069106,1.403715e+09,0.930110,2.165487,1.024433
922,1403715278912143104,0.938216,2.169273,1.043356,0.067404,-0.809726,-0.046721,-0.581049,0.157800,0.072346,...,-0.002229,0.020700,0.07635,-0.012493,0.547732,0.069107,1.403715e+09,0.938216,2.169273,1.043356
932,1403715278962142976,0.945812,2.172850,1.061818,0.067143,-0.809063,-0.046858,-0.581991,0.148173,0.072046,...,-0.002229,0.020700,0.07635,-0.012493,0.547733,0.069108,1.403715e+09,0.945812,2.172850,1.061818
942,1403715279012142848,0.952894,2.176038,1.076596,0.066258,-0.808749,-0.047453,-0.582480,0.135422,0.067486,...,-0.002229,0.020700,0.07635,-0.012493,0.547735,0.069109,1.403715e+09,0.952894,2.176038,1.076596
952,1403715279062142976,0.959404,2.179212,1.086272,0.065887,-0.808885,-0.047710,-0.582312,0.129306,0.065217,...,-0.002229,0.020700,0.07635,-0.012494,0.547736,0.069110,1.403715e+09,0.959404,2.179212,1.086272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28662,1403715417612143104,0.489488,1.968256,0.967014,0.154979,0.780966,-0.241689,0.554671,-0.000866,-0.002883,...,-0.002214,0.020814,0.07628,-0.013988,0.547514,0.067583,1.403715e+09,0.489488,1.968256,0.967014
28672,1403715417662142976,0.489440,1.968178,0.966973,0.154960,0.781015,-0.241701,0.554602,-0.000419,-0.000464,...,-0.002214,0.020814,0.07628,-0.013988,0.547514,0.067583,1.403715e+09,0.489440,1.968178,0.966973
28682,1403715417712142848,0.489410,1.968160,0.966999,0.154918,0.780938,-0.241741,0.554705,-0.000459,0.000289,...,-0.002214,0.020814,0.07628,-0.013988,0.547514,0.067583,1.403715e+09,0.489410,1.968160,0.966999
28692,1403715417762142976,0.489420,1.968117,0.967072,0.154948,0.780900,-0.241712,0.554763,0.000370,-0.002975,...,-0.002214,0.020814,0.07628,-0.013988,0.547514,0.067583,1.403715e+09,0.489420,1.968117,0.967072
